In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#data1=pd.read_csv('bs140513_032310.csv')
data1=pd.read_csv('D:\\DataScienceCompetitions\\AI4Hackathon\\synthetic-data-from-a-financial-payment-system\\bs140513_032310.csv')

In [4]:
data1.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [5]:
#zipcodeOri and zipMerchant doesn't have any predictive power, zero variance(unique value)
data1=data1.drop(columns=['zipcodeOri', 'zipMerchant'])

In [6]:
data1.shape

(594643, 8)

In [7]:
df=data1.groupby('customer').agg({'fraud':'sum','category':'count' }).reset_index().\
rename(columns = {'category':'num_transactions', 'fraud':'num_frauds'})

In [8]:
df['ratio_fraud']=df['num_frauds']/df['num_transactions']

In [9]:
data1=pd.merge(data1, df[['customer', 'ratio_fraud']], on='customer')

In [10]:
data1.head()

,step,customer,age,gender,merchant,category,amount,fraud,ratio_fraud
0,0,'C1093826151','4','M','M348934600','es_transportation',4.55,0,0.0
1,1,'C1093826151','4','M','M855959430','es_hyper',14.38,0,0.0
2,2,'C1093826151','4','M','M348934600','es_transportation',37.21,0,0.0
3,3,'C1093826151','4','M','M348934600','es_transportation',31.63,0,0.0
4,4,'C1093826151','4','M','M348934600','es_transportation',35.86,0,0.0


In [11]:
#dropping customer id
data1=data1.drop(columns=['customer'])

In [12]:
x=list(data1.columns)

In [13]:
x.remove('fraud')

In [14]:
x

['step', 'age', 'gender', 'merchant', 'category', 'amount', 'ratio_fraud']

In [15]:
X=data1.loc[:,x].values
y=data1.iloc[:, 6].values

In [16]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [17]:
labelencoder_X1=LabelEncoder()
X[:,1]=labelencoder_X1.fit_transform(X[:,1])

labelencoder_X2=LabelEncoder()
X[:,2]=labelencoder_X2.fit_transform(X[:,2])

labelencoder_X3=LabelEncoder()
X[:,3]=labelencoder_X3.fit_transform(X[:,3])

labelencoder_X4=LabelEncoder()
X[:,4]=labelencoder_X4.fit_transform(X[:,4])


In [18]:
# Encoding categorical data
onehotencoder = OneHotEncoder(categorical_features=[1,2,3,4])
X = onehotencoder.fit_transform(X).toarray()

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=0)

In [20]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



# DecisionTree

In [20]:
from sklearn.tree import DecisionTreeClassifier
classifier_DT=DecisionTreeClassifier(criterion='entropy', random_state=0)
classifier_DT.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [21]:
y_pred=classifier_DT.predict(X_test)

In [22]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117207,    282],
       [   284,   1156]])

In [23]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9001887765379455

In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117489
          1       0.80      0.80      0.80      1440

avg / total       1.00      1.00      1.00    118929



# Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier
classifier_RF=RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
classifier_RF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [26]:
y_pred=classifier_RF.predict(X_test)

In [27]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117358,    131],
       [   252,   1188]])

In [28]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9119425010000936

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117489
          1       0.90      0.82      0.86      1440

avg / total       1.00      1.00      1.00    118929



# SVM(Kernal=Poly)

In [30]:
from sklearn.svm import SVC
classifier_pl=SVC(kernel='poly', random_state=0)
classifier_pl.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [31]:
y_pred=classifier_pl.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117345,    144],
       [   295,   1145]])

In [33]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8969566211163031

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117489
          1       0.89      0.80      0.84      1440

avg / total       1.00      1.00      1.00    118929



# ANN

In [35]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [36]:
classifier = Sequential()

In [37]:
len(X_train[1])

80

In [43]:
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 80))
#classifier.add(Dropout(p = 0.2))
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
#classifier.add(Dropout(p = 0.2))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [44]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                1296      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
_________________________________________________________________
dense_5 (Dense)              (None, 16)                32        
_________________________________________________________________
dense_6 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total para

In [45]:
classifier.fit(X_train, y_train, batch_size = 100, epochs = 20)

Epoch 1/20
475714/475714 [==============================] - 36s 76us/step - loss: 0.0796 - acc: 0.9877
Epoch 2/20
475714/475714 [==============================] - 32s 68us/step - loss: 0.0656 - acc: 0.9879
Epoch 3/20
475714/475714 [==============================] - 41s 86us/step - loss: 0.0656 - acc: 0.9879
Epoch 4/20
475714/475714 [==============================] - 39s 81us/step - loss: 0.0656 - acc: 0.9879
Epoch 5/20
475714/475714 [==============================] - 41s 86us/step - loss: 0.0656 - acc: 0.9879
Epoch 6/20
475714/475714 [==============================] - 32s 67us/step - loss: 0.0656 - acc: 0.9879
Epoch 7/20
475714/475714 [==============================] - 40s 84us/step - loss: 0.0656 - acc: 0.9879
Epoch 8/20
475714/475714 [==============================] - 36s 77us/step - loss: 0.0656 - acc: 0.9879
Epoch 9/20
475714/475714 [==============================] - 40s 83us/step - loss: 0.0656 - acc: 0.9879
Epoch 10/20
475714/475714 [==============================] - 39s 82us/ste

In [46]:
y_pred=classifier.predict(X_test)

In [47]:
y_pred = [round(x[0]) for x in y_pred]

In [48]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117489,      0],
       [  1440,      0]])

In [49]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.5

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99    117489
          1       0.00      0.00      0.00      1440

avg / total       0.98      0.99      0.98    118929



/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# XGBoost

In [21]:
from xgboost import XGBClassifier
classifier_xg = XGBClassifier()
classifier_xg.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [22]:
y_pred=classifier_xg.predict(X_test)

C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117332,    157],
       [   282,   1158]], dtype=int64)

In [24]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.901415185676957

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117489
          1       0.88      0.80      0.84      1440

avg / total       1.00      1.00      1.00    118929



# Cross Validation

In [26]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier_xg, X=X_train, y=y_train, cv =10)
accuracies.mean()


C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\labe

0.9962918895495159

In [27]:
accuracies

array([0.99630034, 0.99634239, 0.9961532 , 0.99642647, 0.99640537,
       0.99594291, 0.99642639, 0.99625822, 0.99615312, 0.99651048])

# SVM Kernal=Linear

In [51]:
from sklearn.svm import SVC
classifier=SVC(kernel='linear', random_state=0)
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [52]:
y_pred=classifier.predict(X_test)

In [53]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117355,    134],
       [   296,   1144]])

In [54]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8966519560696461

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117489
          1       0.90      0.79      0.84      1440

avg / total       1.00      1.00      1.00    118929



# Logistic Regression

In [56]:
from sklearn.linear_model import LogisticRegression
classifier_lr=LogisticRegression(random_state=0)
classifier_lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [57]:
y_pred=classifier_lr.predict(X_test)

In [58]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117348,    141],
       [   319,   1121]])

In [59]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8886360549356394

In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117489
          1       0.89      0.78      0.83      1440

avg / total       1.00      1.00      1.00    118929



# KNN Classifier

In [61]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn=KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [63]:
y_pred=classifier_knn.predict(X_test)

In [64]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117315,    174],
       [   315,   1125]])

In [66]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8898845051451625

In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117489
          1       0.87      0.78      0.82      1440

avg / total       1.00      1.00      1.00    118929



# Naive Bayes

In [68]:
from sklearn.naive_bayes import GaussianNB
classifier_NB=GaussianNB()
classifier_NB.fit(X_train, y_train)

GaussianNB(priors=None)

In [69]:
y_pred=classifier_NB.predict(X_test)

In [70]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[111481,   6008],
       [     1,   1439]])

In [71]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9740844266981022

In [72]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      0.95      0.97    117489
          1       0.19      1.00      0.32      1440

avg / total       0.99      0.95      0.97    118929

